# TDE Encryption & Renewal

- [Rotating TDE Certificate](https://matthewmcgiffen.com/2023/02/06/rotating-a-certificate-used-by-tde/)
- [Create Certificate](https://learn.microsoft.com/en-us/sql/t-sql/statements/create-certificate-transact-sql?view=sql-server-ver16)
- [Change expiry date of TDE certificate of User Database without changing Thumbprint](https://dba.stackexchange.com/a/213294)

## Check Encryption Progress/State on DbServer

In [1]:
USE [master];
-- Check Encryption Status
SELECT  --[sql_instance] = '$SqlInstanceToEncrypt',
        [database_name] = d.name, encryption_state_desc, percent_complete, --encryptor_thumbprint,
		[key_algorithm] = ek.key_algorithm + '_'+ convert(varchar,ek.key_length), 
		[certificate_name] = c.name, [certificate_expiry_date] = c.expiry_date, 
		[certificate_expiry_days] = case when c.expiry_date <= getdate() then 0 else datediff(day,getdate(),c.expiry_date) end,
		[certificate_subject] = c.subject,
		[certificate_start_date] = c.start_date,  c.pvt_key_last_backup_date,
		[encryption_key_create_date] = regenerate_date --encryption_scan_state_desc
FROM sys.databases d
left join sys.dm_database_encryption_keys  ek
	on ek.database_id = d.database_id
left join sys.certificates c
	on c.thumbprint = ek.encryptor_thumbprint
    and c.pvt_key_encryption_type_desc = 'ENCRYPTED_BY_MASTER_KEY'
WHERE 1=1
and d.state_desc = 'ONLINE' 
and is_read_only = 0 
and d.database_id > 4
and d.source_database_id is null
go

(2 rows affected)

Total execution time: 00:00:00.042

database_name,encryption_state_desc,percent_complete,key_algorithm,certificate_name,certificate_expiry_date,certificate_expiry_days,certificate_subject,certificate_start_date,pvt_key_last_backup_date,encryption_key_create_date
DBATools,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
DBA,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


## Databases to Encrypt

In [2]:
/* Get list of databases to encrypt: $ClientName */
select [database_name] = d.name, d.state_desc, d.is_read_only, 
		d.is_in_standby, d.is_published, d.is_subscribed,
		rs.is_local, rs.replica_server_name, rs.role_desc
		,dm.mirroring_role_desc
from sys.databases d 
outer apply (select top 1 rs.is_local, ar.replica_server_name, rs.role_desc
			from sys.availability_replicas ar
			join sys.dm_hadr_availability_replica_states rs
			on rs.replica_id = ar.replica_id and rs.group_id = ar.group_id
			where ar.replica_id = d.replica_id
			) rs
outer apply (select top 1 dm.mirroring_role_desc
			from sys.database_mirroring dm
			where dm.database_id = d.database_id
				and dm.mirroring_role_desc is not null
			) dm
where 1=1
    and d.state_desc = 'ONLINE' 
    and is_read_only = 0 
    and d.database_id > 4
	--and d.is_encrypted = 0
	and d.source_database_id is null

(2 rows affected)

Total execution time: 00:00:00.060

database_name,state_desc,is_read_only,is_in_standby,is_published,is_subscribed,is_local,replica_server_name,role_desc,mirroring_role_desc
DBATools,ONLINE,0,0,0,0,1,AGHOST-1B,PRIMARY,NULL
DBA,ONLINE,0,0,0,0,NULL,NULL,NULL,NULL


## Create table on Inventory

In [ ]:
--drop table dbo.tde_implementation_details
create table dbo.tde_implementation_details
(
	srv_name varchar(125) not null,
	at_server_name varchar(128) not null,
	server_name varchar(125) not null,
	domain varchar(125) not null,
	server_host_name varchar(125) not null,
	encrypted_databases varchar(125) null,
	total_database_count smallint not null,
	encryption_start_time datetime2 not null default sysdatetime(),
	encryption_end_time datetime2 null,
	local_backup_directory varchar(500) not null,
	remote_backup_directory varchar(500) null,
	certificate_name varchar(125) not null,
	certificate_subject varchar(500) null,
	encryption_password varchar(500) null,
	certificate_file_path varchar(500) null,
	master_key_path varchar(500) null,
	private_key_path varchar(500) null,
	files_copied_to_remote bit not null default 0,
	collection_time datetime2 not null default sysdatetime(),
	collected_by varchar(125) not null default suser_name()

	,constraint pk__tde_implementation_details primary key clustered (srv_name, server_name, server_host_name, certificate_name)
);
go

In [ ]:
-- create view - get TDE Certificate Ages
create or alter view dbo.dba_tde_cert_age 
as
with cte_latest as (
	select tde.srv_name, encryption_start_time = max(tde.encryption_start_time), 
			end_date = DATEADD(year,max(age_limit),max(tde.encryption_start_time))
	from dbo.tde_implementation_details tde
	cross apply (select [age_limit] = case when tde.encryption_start_time < '2025-01-01' then 1 else 5 end) age
	group by tde.srv_name
)
select *, remaining_days = case when end_date < getdate() then 0 else datediff(day,GETDATE(),end_date) end
from cte_latest
go


## Certificate Details from Inventory

In [ ]:
-- Save Encryption Details of [192.168.100.11]
-- Execute below on [OfficeInventory].[DBA]
if not exists (select * from dbo.tde_implementation_details where srv_name = '192.168.100.11' and server_name = 'SqlPractice' and server_host_name = 'SqlPractice' and certificate_name = 'SqlPractice__Certificate')
begin
	insert dbo.tde_implementation_details
    ( [srv_name], [at_server_name], [server_name], [domain], [server_host_name], [encrypted_databases], [total_database_count], [encryption_start_time], [encryption_end_time], [local_backup_directory], [remote_backup_directory], [certificate_name], [certificate_subject], [encryption_password], [certificate_file_path], [master_key_path], [private_key_path], [files_copied_to_remote] )
    select	[srv_name] ='192.168.100.11', 
		    [at_server_name] = 'SqlPractice', 
		    [server_name] = 'SqlPractice', 
		    [domain] = 'LAB', 
		    [server_host_name] = 'SqlPractice', 
		    [encrypted_databases] = 1, 
		    [total_database_count] = 1, 
		    [encryption_start_time] = SYSDATETIME(), 
		    [encryption_end_time] = null, 
		    [local_backup_directory] = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\', 
		    [remote_backup_directory] = 'D:\Work', 
		    [certificate_name] = 'SqlPractice__Certificate', 
		    [certificate_subject] = 'SqlPractice Certificate', 
		    [encryption_password] = null, 
		    [certificate_file_path] = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__Certificate.crt', 
		    [master_key_path] = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__master_key.key', 
		    [private_key_path] = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__private_key.pvk', 
		    [files_copied_to_remote] = 0;
    
    print 'TDE Entry added for [192.168.100.11].'
end
else
begin
    print 'TDE Entry already exists for [192.168.100.11].'
    select * from dbo.tde_implementation_details where srv_name = '192.168.100.11' and server_name = 'SqlPractice' and server_host_name = 'SqlPractice';
end
GO

## Fetch/Save Encryption Key Password

In [ ]:
-- Execute below on [OfficeInventory].[DBA]
-- Get Encryption Password for [<SomeServerName>]
select	top 1 server_ip, server_name, [user_name], 
		[password] = cast(DecryptByPassPhrase(cast(salt as varchar),password_hash ,1, server_ip) as varchar)
from dbo.credential_manager cm
where cm.server_ip = '<SomeServerName>'
and user_name = 'master key'
GO



-- Execute below on [OfficeInventory].[DBA]
-- Save Encryption Key on Credential Manager
exec dbo.usp_add_credential
			@server_ip = '<SomeServerName>',
			--@server_name = 'SqlPractice',
			@user_name = 'master key',
			@password_string = '<SomeVeryStrongPassword>',
			@remarks = 'Database Master Key';
GO

## Script to Restore Certificate

In [ ]:
USE [master];
-- For Restore Scenario (AG/Mirroring/LogShipping/Backup-Restore)
create master key encryption by /* Step 1: Unique to Destination Server */
	password = '<<Some Very Strong Password Here. Unique to each server>>';

-- For Restore Scenario (AG/Mirroring/LogShipping/Backup-Restore)
create certificate [SqlPractice__Certificate] /* Step 2: Details similar to Source Server */
	from file = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__Certificate.crt'
	with private key (
		file = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__private_key.pvk',
		decryption by password = 'SomeStrongPasswordHere'
	);
GO

## TDE Sample Code (Execute on Inventory + Db Server)

In [ ]:
-- ***** Implement Transparent Data Encryption (TDE) on server [192.168.100.11]

-- Execute below on [OfficeInventory].[DBA]
-- Get Encryption Password for [192.168.100.11]
select	top 1 server_ip, server_name, [user_name], 
		[password] = cast(DecryptByPassPhrase(cast(salt as varchar),password_hash ,1, server_ip) as varchar)
from dbo.credential_manager cm
where cm.server_ip = '192.168.100.11'
and user_name = 'master key'
GO

-- Execute below on [OfficeInventory].[DBA]
-- Save Encryption Key on Credential Manager
exec dbo.usp_add_credential
			@server_ip = '192.168.100.11',
			@server_name = 'SqlPractice',
			@user_name = 'master key',
			@password_string = 'SomeStrongPasswordHere',
			@remarks = 'Database Master Key';
GO

-- Create master key
if not exists (select * from sys.symmetric_keys where name LIKE '%DatabaseMasterKey%')
begin    
	exec ('use master; create master key encryption by password = ''SomeStrongPasswordHere'';');
    print 'DMK created';
end
else
    print 'DMK exists';
GO

-- Create certificate
declare @sql nvarchar(max);
declare @thumbprint varbinary(64);

select @thumbprint = c.thumbprint from sys.certificates c 
	where issuer_name = 'SqlPractice Certificate' and name = 'SqlPractice__Certificate';

if @thumbprint is null
begin
	set @sql = 'use master; create certificate [SqlPractice__Certificate] with subject = ''SqlPractice Certificate'', EXPIRY_DATE = ''20241218'';';
	exec (@sql);
    print 'certificate created';
end
else
	print 'Certificate already exists';
GO

declare @sql nvarchar(max);
declare @thumbprint varbinary(64);

use [master];
-- Get certificate thumbprint
select @thumbprint = c.thumbprint from sys.certificates c 
	where issuer_name = 'SqlPractice Certificate' and name = 'SqlPractice__Certificate';

use [DBA_Admin];
-- Create encryption key
if not exists (select * from sys.dm_database_encryption_keys dek where dek.database_id = DB_ID()
					and dek.encryptor_type = 'CERTIFICATE' and dek.key_length = 128 
					and dek.encryptor_thumbprint = @thumbprint )
begin
	set @sql = 'use '+quotename(DB_NAME())+'; '
                +char(9)+'create database encryption key with algorithm = aes_128 '
	            +char(9)+char(9)+'encryption by server certificate [SqlPractice__Certificate];';
	exec (@sql);
    print 'Encryption Key created on [DBA_Admin].'
end
else
	print 'Database Encryption key exists';
GO

-- Backup Certificate & Master Key
use master;

declare @sql nvarchar(max);

exec xp_create_subdir 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\';

/* **** If Cleanup of Files is required ****
DECLARE @cmd NVARCHAR(MAX);

SET @cmd = 'xp_cmdshell ''del "C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__master_key.key"''';
EXEC (@cmd); -- remove master key file

SET @cmd = 'xp_cmdshell ''del "C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__Certificate.crt"'''; 
EXEC (@cmd); -- remove certificate file

SET @cmd = 'xp_cmdshell ''del "C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__private_key.pvk"'''; 
EXEC (@cmd); -- remove private key file
*/

-- Backup master key
set @sql = 'use master; backup master key to file = ''C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__master_key.key'' encryption by password = ''SomeStrongPasswordHere''';
exec (@sql);

-- Backup certificate
set @sql = 'use master; backup certificate [SqlPractice__Certificate]
	to file = ''C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__Certificate.crt''
	with private key (
		file = ''C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__private_key.pvk'',
		encryption by password = ''SomeStrongPasswordHere''
	)';
exec (@sql);

/* IMPORTANT: COPY these files to some secure location Immediately before proceeding */

GO

-- Enable Encryption
use [master]; alter database [DBA_Admin] set encryption on;
GO


/* ************************************************** */
-- Save Encryption Details of [192.168.100.11]
-- Execute below on [OfficeInventory].[DBA]
if not exists (select * from dbo.tde_implementation_details where srv_name = '192.168.100.11' and server_name = 'SqlPractice' and server_host_name = 'SqlPractice')
begin
	insert dbo.tde_implementation_details
    ( [srv_name], [at_server_name], [server_name], [domain], [server_host_name], [encrypted_databases], [total_database_count], [encryption_start_time], [encryption_end_time], [local_backup_directory], [remote_backup_directory], [certificate_name], [certificate_subject], [encryption_password], [certificate_file_path], [master_key_path], [private_key_path], [files_copied_to_remote] )
    select	[srv_name] ='192.168.100.11', 
		    [at_server_name] = 'SqlPractice', 
		    [server_name] = 'SqlPractice', 
		    [domain] = 'Lab', 
		    [server_host_name] = 'SqlPractice', 
		    [encrypted_databases] = 1, 
		    [total_database_count] = 1, 
		    [encryption_start_time] = SYSDATETIME(), 
		    [encryption_end_time] = null, 
		    [local_backup_directory] = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\', 
		    [remote_backup_directory] = 'D:\Work', 
		    [certificate_name] = 'SqlPractice__Certificate', 
		    [certificate_subject] = 'SqlPractice Certificate', 
		    [encryption_password] = null, 
		    [certificate_file_path] = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__Certificate.crt', 
		    [master_key_path] = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__master_key.key', 
		    [private_key_path] = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__private_key.pvk', 
		    [files_copied_to_remote] = 0;
    
    print 'TDE Entry added for [192.168.100.11].'
end
else
    print 'TDE Entry already exists for [192.168.100.11].'
GO


/* ************************************************** */
USE [master];
-- Check Encryption Status
SELECT DB_NAME(database_id) as dbName, encryption_state_desc, percent_complete,
		encryptor_thumbprint,
		[encryption_key_create_date] = regenerate_date,
		encryption_scan_state_desc,
		*
FROM sys.dm_database_encryption_keys 
WHERE database_id > 4
--and DB_NAME(database_id) in ('')
GO


/* ************************************************** */
USE [master];
-- For Restore Scenario (AG/Mirroring/LogShipping/Backup-Restore)
create master key encryption by /* Step 1: Unique to Destination Server */
	password = '<<Some Very Strong Password Here. Unique to each server>>';

-- For Restore Scenario (AG/Mirroring/LogShipping/Backup-Restore)
create certificate [SqlPractice__Certificate] /* Step 2: Details similar to Source Server */
	from file = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__Certificate.crt'
	with private key (
		file = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Log\SqlPractice__private_key.pvk',
		decryption by password = 'SomeStrongPasswordHere'
	);
GO

## Mailer for Expiring Certificates

In [ ]:
use DBA
go

CREATE OR ALTER PROCEDURE dbo.sp_Send_TDE_Cert_Expiry_Alert
	@pJobName nvarchar(500) = '(dba) Send-TdeCertExpiryAlert'
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @EmailBody NVARCHAR(MAX);
    DECLARE @HTMLHeader NVARCHAR(MAX);
    DECLARE @RowCount INT;
	DECLARE @Recepient NVARCHAR(500);

	select @Recepient = P.param_value from dbo.sma_params p where p.param_key = 'dba_team_email_id';

    -- Temporary Table to Store Query Results
    CREATE TABLE #CertExpiryResults (
        srv_name NVARCHAR(255),
        encryption_start_time DATETIME,
        end_date DATETIME,
        remaining_days INT
    );

    -- Fetch Certificates Near Expiry
    INSERT INTO #CertExpiryResults (srv_name, encryption_start_time, end_date, remaining_days)
    SELECT srv_name, encryption_start_time, end_date, remaining_days
    FROM DBA_Inventory.dbo.dba_tde_cert_age a
    WHERE a.remaining_days <= 90
    ORDER BY a.remaining_days ASC, srv_name;

    -- Check if Data Exists
    SELECT @RowCount = COUNT(*) FROM #CertExpiryResults;

    IF @RowCount > 0
    BEGIN
        -- Build HTML Email Body
        SET @HTMLHeader = N'
            <h2>TDE Certificate Expiry Alert</h2>
            <p>The following certificates are expiring within the next 90 days:</p>
            <table border="1" cellpadding="5" cellspacing="0" style="border-collapse: collapse;">
                <tr>
                    <th>Server Name</th>
                    <th>Encryption Start Time</th>
                    <th>End Date</th>
                    <th>Remaining Days</th>
                </tr>';

        SELECT @EmailBody = (
            SELECT 
                '<tr>' +
                '<td>' + srv_name + '</td>' +
                '<td>' + CONVERT(NVARCHAR(50), encryption_start_time, 120) + '</td>' +
                '<td>' + CONVERT(NVARCHAR(50), end_date, 120) + '</td>' +
                '<td>' + CAST(remaining_days AS NVARCHAR(10)) + '</td>' +
                '</tr>'
            FROM #CertExpiryResults
			ORDER BY remaining_days asc, srv_name
            FOR XML PATH(''), TYPE
        ).value('.', 'NVARCHAR(MAX)');

        SET @EmailBody = @HTMLHeader + @EmailBody + '</table>';

		SET @EmailBody = @EmailBody + '<br><br><p>Regards,<br>'+QUOTENAME(@pJobName)+'<br></p>';

        -- Send Email
        EXEC msdb.dbo.sp_send_dbmail
            --@profile_name = 'DBA_Email_Profile', -- Replace with your Database Mail profile name
            @recipients = @Recepient,
			--@recipients = 'sqlagentservice@gmail.com',
            @subject = 'TDE Certificate Expiry Alert',
            @body = @EmailBody,
            @body_format = 'HTML';

        PRINT 'Email sent successfully.';
    END
    ELSE
    BEGIN
        PRINT 'No certificates are expiring within the next 90 days.';
    END

    -- Cleanup
    DROP TABLE #CertExpiryResults;
END;
GO

EXEC dbo.sp_Send_TDE_Cert_Expiry_Alert;
go

## PowerShell Code to Implement TDE
It creates new certificate if existing expired.

### Encrypt-SQLDatabases.ps1

In [ ]:
[CmdletBinding()]
Param (
    [Parameter(Mandatory=$false)]
    [String]$SqlInstanceToEncrypt = 'SqlPractice',

    [Parameter(Mandatory=$false)]
    [String[]]$DatabasesToEncrypt,

    [Parameter(Mandatory=$false)]
    [String]$DbaDatabase = 'DBA',

    [Parameter(Mandatory=$false)]
    [String]$InventoryServer = 'SQLMonitor',

    [Parameter(Mandatory=$false)]
    [String]$InventoryDatabase = 'DBA',

    [Parameter(Mandatory=$false)]
    [String]$CredentialManagerServer = 'SQLMonitor',

    [Parameter(Mandatory=$false)]
    [String]$CredentialManagerDatabase = 'DBA',

    [Parameter(Mandatory=$false)]
    [PSCredential]$SqlCredential = $personal,

    [Parameter(Mandatory=$false)]
    [PSCredential]$WindowsCredential,

    [Parameter(Mandatory=$false)]
    [String]$LocalBackupDirectory,

    [Parameter(Mandatory=$false)]
    [String]$RemoteBackupDirectory = 'E:\Work',

    [Parameter(Mandatory=$false)]
    [String]$EncryptionPassword,

    [Parameter(Mandatory=$false)]
    [bool]$DryRun = $true,

    [Parameter(Mandatory=$false)]
    [bool]$ScriptOut = $false,

    [Parameter(Mandatory=$false)]
    [bool]$RenewCertificate = $true,

    [Parameter(Mandatory=$false)]
    [int]$ExpiryDaysThreshold = 365, # if expiring in days

    [Parameter(Mandatory=$false)]
    [int]$ExpiryYearsToSetForCertificate = 5 # set expiry date for certificate from today
)

cls
$startTime = Get-Date
$ErrorActionPreference = "Stop"
$ClientName = "Encrypt-SQLDatabases.ps1"

$scriptOutfile = New-TemporaryFile
$horizontalLine = "`n/* " + ('*' * 50) + " */"

"-- ***** Implement Transparent Data Encryption (TDE) on server [$SqlInstanceToEncrypt]`n`n" | Out-File -FilePath $scriptOutfile

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'START:', "Working on server [$SqlInstanceToEncrypt]." | Write-Host -ForegroundColor Yellow

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "[Connect-DbaInstance] Create connection for InventoryServer '$InventoryServer'.."
$conInventoryServer = Connect-DbaInstance -SqlInstance $InventoryServer -Database $InventoryDatabase -ClientName $ClientName `
                                                    -SqlCredential $SqlCredential -TrustServerCertificate -ErrorAction Stop

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "[Connect-DbaInstance] Create connection for CredentialManagerServer '$CredentialManagerServer'.."
$conCredentialManagerServer = Connect-DbaInstance -SqlInstance $CredentialManagerServer -Database $CredentialManagerDatabase -ClientName $ClientName `
                                                    -SqlCredential $SqlCredential -TrustServerCertificate -ErrorAction Stop

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "[Connect-DbaInstance] Create connection for SqlInstanceToEncrypt '$SqlInstanceToEncrypt'.."
$conSqlInstanceToEncrypt = Connect-DbaInstance -SqlInstance $SqlInstanceToEncrypt -Database master -ClientName $ClientName `
                                                    -SqlCredential $SqlCredential -TrustServerCertificate -ErrorAction Stop

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "[Invoke-DbaQuery] Get basic details of '$SqlInstanceToEncrypt'.."
$sqlBasicDetails = @"
use master;
select	srv_name = '$SqlInstanceToEncrypt',
		at_server_name = @@SERVERNAME,
		server_name = convert(varchar,SERVERPROPERTY('ServerName')),
		domain = default_domain(),
		server_host_name = SERVERPROPERTY('ComputerNamePhysicalNetBIOS'),
		total_database_count = (select count(*) from sys.databases d where d.state_desc = 'ONLINE' and is_read_only = 0 and d.database_id > 4),
		error_log_file = SERVERPROPERTY('ErrorLogFileName'),
        master_key_exists = convert(bit,case when exists (select * from sys.symmetric_keys where name LIKE '%DatabaseMasterKey%') then 1 else 0 end)
"@
$resultBasicDetails = @()
$resultBasicDetails += Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlBasicDetails

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "[Invoke-DbaQuery] Get existing encryption status per database.."
$sqlDbEncryptionDetails = @"
USE [master];
-- Check Encryption Status
SELECT  [sql_instance] = '$SqlInstanceToEncrypt',
        [database_name] = d.name, encryption_state_desc, percent_complete, --encryptor_thumbprint,
		[key_algorithm] = ek.key_algorithm + '_'+ convert(varchar,ek.key_length), 
		[certificate_name] = c.name, [certificate_expiry_date] = c.expiry_date, 
		[certificate_expiry_days] = case when c.expiry_date is null then null
                                         when c.expiry_date <= getdate() then 0 
                                         else datediff(day,getdate(),c.expiry_date) 
                                         end,
		[certificate_subject] = c.subject,
		[certificate_start_date] = c.start_date,  c.pvt_key_last_backup_date,
		[encryption_key_create_date] = regenerate_date --encryption_scan_state_desc
FROM sys.databases d
left join sys.dm_database_encryption_keys  ek
	on ek.database_id = d.database_id
left join sys.certificates c
	on c.thumbprint = ek.encryptor_thumbprint
    and c.pvt_key_encryption_type_desc = 'ENCRYPTED_BY_MASTER_KEY'
WHERE 1=1
and d.state_desc = 'ONLINE' 
and is_read_only = 0 
and d.database_id > 4
and d.source_database_id is null
"@
$resultDbEncryptionDetails = @()
$resultDbEncryptionDetails += Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlDbEncryptionDetails

#$resultBasicDetails | ogv
#$resultDbEncryptionDetails | ogv

$serverName = $resultBasicDetails.server_name
$serverNameDeSensitized = $serverName.Replace('\','_')
$atServerName = $resultBasicDetails.at_server_name
$domain = $resultBasicDetails.domain
$serverHostName = $resultBasicDetails.server_host_name
$totalDatabaseCount = $resultBasicDetails.total_database_count
$errorLogFile = $resultBasicDetails.error_log_file
$errorLogFileDirectory = $(Split-Path $errorLogFile)
$masterKeyExists = $resultBasicDetails.master_key_exists
$saveEncryptionPassword = $false

# get current & previous certificate
[String]$certificateSubject = $serverName+' Certificate'
[string]$currentCertificateName = $null
[int]$currentCertificateAgeLeft = $null
[string]$previousCertificateName = $null
[bool]$hasCertificateExpired = $false
[bool]$createNewCertificate = $true
[int]$pendingDbCount = 0
[int]$encryptedDbCount = 0
[int]$expiredDbCount = 0

# If there are user databases, then extract info
if($resultDbEncryptionDetails.Count -gt 0)
{
    $currentCertificateName = $resultDbEncryptionDetails | Where-Object {$_.certificate_subject -eq $certificateSubject} | `
                                Sort-Object -Property certificate_start_date -Descending | `
                                Select-Object -ExpandProperty certificate_name -First 1
    
    if(-not [String]::($currentCertificateName)) {
        $currentCertificateAgeLeft = $resultDbEncryptionDetails | Where-Object {$_.certificate_name -eq $currentCertificateName} | `
                                    Select-Object -ExpandProperty certificate_expiry_days -First 1
        $previousCertificateName = $resultDbEncryptionDetails | `
                                    Where-Object { ($_.certificate_subject -eq $certificateSubject) -and ($_.certificate_name -notin @($currentCertificateName)) } | `
                                    Sort-Object -Property certificate_start_date -Descending | `
                                    Select-Object -ExpandProperty certificate_name -First 1
    }

    $pendingDbCount = ($resultDbEncryptionDetails | Where-Object {[String]::IsNullOrEmpty($_.encryption_state_desc)}).Count
    $encryptedDbCount = ($resultDbEncryptionDetails | Where-Object {-not [String]::IsNullOrEmpty($_.encryption_state_desc)}).Count
    $expiredDbCount = ($resultDbEncryptionDetails | Where-Object { (-not [String]::IsNullOrEmpty($_.certificate_expiry_days)) -and ($_.certificate_expiry_days -le $ExpiryDaysThreshold) }).Count
}

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "`$encryptedDbCount = $encryptedDbCount"
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "`$currentCertificateAgeLeft = $currentCertificateAgeLeft"
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "`$ExpiryDaysThreshold = $ExpiryDaysThreshold"
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "`$expiredDbCount = $expiredDbCount"

# has certificates expired?
if( ($encryptedDbCount -gt 0) -and ($currentCertificateAgeLeft -lt $ExpiryDaysThreshold) -and ($expiredDbCount -gt 0) ) {
    $hasCertificateExpired = $true
}
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "`$hasCertificateExpired = $hasCertificateExpired"


# return if no encryption, or no certificate renewal issue
if( $pendingDbCount -eq 0 -and $hasCertificateExpired -eq $false -and $expiredDbCount -eq 0 ) {
    "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'RESULT:', "No database pending for encryption."

    if(-not $ScriptOut) {
        return
    }
}

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Either there are dbs to be encrypted, or certificate renewal is required."

# get new certificate
[String]$certificateName = $null
if($hasCertificateExpired) {
    "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "currentCertificateName: '$currentCertificateName'"    
    "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Certificate renewal is needed."

    # if certificate name has numbering
    if($currentCertificateName -match ".+__Certificate__(?'cert_no'\d+)*$") {
        $certificateName = $serverNameDeSensitized+'__Certificate'+'__'+$( ([int]$Matches['cert_no'])+1 )
    }
    else {
        $certificateName = $serverNameDeSensitized+'__Certificate'+'__2'
    }
}
else {
    if(-not [String]::IsNullOrEmpty($currentCertificateName)) {
        $certificateName = $currentCertificateName
    }
    else {
        $certificateName = $serverNameDeSensitized+'__Certificate'
    }
}

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Configure Local Backup Directory.."
$backupDirectory = $LocalBackupDirectory
if([String]::IsNullOrEmpty($backupDirectory)) {
    $backupDirectory = $errorLogFileDirectory
}

# If Errorlog path is not found, then use sp_readerrorlog to detect same
if([String]::IsNullOrEmpty($backupDirectory)) {
    $sqlGetErrorLog = "exec sp_readerrorlog 0,1,'Log\ERRORLOG','-e'"
    $resultGetErrorLog = @()
    $resultGetErrorLog += Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlGetErrorLog
    if($resultGetErrorLog[0].Text -match "\s+-e\s(?'log_directory'.+)\\ERRORLOG") {
        $backupDirectory = $Matches['log_directory']
    }
}

if(-not $backupDirectory.EndsWith('\')) {
    $backupDirectory = $backupDirectory + '\'
}


# Get Encryption Password if present in CM
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Fetch Encryption Password from Credential Manager.."
$sqlFetchPassword = @"
    -- Get Encryption Password for [$SqlInstanceToEncrypt]
select	top 1 server_ip, server_name, [user_name], 
		[password] = cast(DecryptByPassPhrase(cast(salt as varchar),password_hash ,1, server_ip) as varchar)
from dbo.credential_manager cm
where cm.server_ip = '$SqlInstanceToEncrypt'
and user_name = 'master key'
"@

"`n`n-- Execute below on [$CredentialManagerServer].[$CredentialManagerDatabase]`n" + $sqlFetchPassword + "`nGO`n" | Out-File -FilePath $scriptOutfile -Append
$resultFetchPassword = @()
$resultFetchPassword += Invoke-DbaQuery -SqlInstance $conCredentialManagerServer -Database $CredentialManagerDatabase -Query $sqlFetchPassword

# If password does not exist in credential manager
if($resultFetchPassword.Count -eq 0) {
    $saveEncryptionPassword = $true
    "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Password not found in Credential Manager."

    if([String]::IsNullOrEmpty($EncryptionPassword)) 
    {
        "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "EncryptionPassword parameter is null."

        if($masterKeyExists) {
            "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'ERROR:', "Master Key on $SqlInstanceToEncrypt exists."
            "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'ERROR:', "But existing Encryption Password not found in Credential Manager or Parameter."

            $sql4Debugging = @"
select * from sys.symmetric_keys where name LIKE '%DatabaseMasterKey%';
select name, state_desc, is_encrypted, is_master_key_encrypted_by_server 
	from sys.databases where name = 'master';

-- encrypt master key by Password
alter master key add encryption by password = '<<<YouMasterDbMasterKeyPasswordHere>>>';
"@
            "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'DEBUG:', "Following code can be used for debugging-`n$sql4Debugging`n"
            "Kindly rectify above error" | Write-Error
        }
        else {
            "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Generate Encryption Password.."
            $EncryptionPassword = -Join("ABCDabcd&@#$%1234".tochararray() | Get-Random -Count 25 | % {[char]$_})
        }
    }
}
else {
    "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Encryption Password found in Credential Manager."

    if( (-not [String]::IsNullOrEmpty($EncryptionPassword)) -and ($EncryptionPassword -ne $resultFetchPassword[0].password) ) {
        "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'ERROR:', "Encryption Password in Credential Manager & Parameter value does not match." | Write-Host -ForegroundColor Red
        "Kindly rectify above error" | Write-Error
    }

    if( [String]::IsNullOrEmpty($EncryptionPassword) ) {
        $EncryptionPassword = $resultFetchPassword[0].password
    }
}


# Save Encryption Password if newly generated
if($saveEncryptionPassword -and (-not [String]::IsNullOrEmpty($EncryptionPassword)) ) 
{
    "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Save Encryption Password in Credential Manager.."
    $params = @{
                    server_ip = $SqlInstanceToEncrypt
                    server_name = $serverName
                    user_name = 'master key'
                    password_string = $EncryptionPassword
                    remarks = 'Database Master Key'
            }
    Invoke-DbaQuery -SqlInstance $conCredentialManagerServer -Database $CredentialManagerDatabase `
            -Query usp_add_credential -SqlParameter $params -CommandType StoredProcedure -MessagesToOutput -EnableException
}
elseif ($saveEncryptionPassword -and [String]::IsNullOrEmpty($EncryptionPassword) ) {
    "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'ERROR:', "Encryption Password could not be fetched or initialized." | Write-Host -ForegroundColor Red
        "Kindly rectify above error" | Write-Error
}

$sqlAddEncryptionPasswordOnCredentialManager = @"
    -- Save Encryption Key on Credential Manager
exec dbo.usp_add_credential
			@server_ip = '$SqlInstanceToEncrypt',
			@server_name = '$serverName',
			@user_name = 'master key',
			@password_string = '$EncryptionPassword',
			@remarks = 'Database Master Key';
"@
"`n`n-- Execute below on [$CredentialManagerServer].[$CredentialManagerDatabase]`n" + $sqlAddEncryptionPasswordOnCredentialManager + "`nGO`n`n" | Out-File -FilePath $scriptOutfile -Append


# Initialize Derived Variables
$certificatePath = $backupDirectory + $certificateName + '.crt'
[String]$expiredCertificatePath = $null
if($hasCertificateExpired) {
    $expiredCertificatePath = $backupDirectory + $currentCertificateName + '.crt'
}

$masterKeyPath = $backupDirectory + $serverNameDeSensitized + '__master_key.key'
$privateKeyPath = $backupDirectory + $serverNameDeSensitized + '__private_key.pvk'


# Create Master Key on Target
$sqlCreateMasterKey = @"
-- Create master key on [$SqlInstanceToEncrypt]
use master;
if not exists (select * from sys.symmetric_keys where name LIKE '%DatabaseMasterKey%')
begin    
	exec ('use master; create master key encryption by password = ''$EncryptionPassword'';');
    print 'DMK created';
end
else
    print 'DMK exists';
"@

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Create Master Key.."
$sqlCreateMasterKey + "`nGO`n`n" | Out-File -FilePath $scriptOutfile -Append
if($DryRun) {
    $sqlCreateMasterKey | Write-Host -ForegroundColor Magenta
}
else {
    Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlCreateMasterKey `
                    -MessagesToOutput -EnableException | Write-Host -ForegroundColor Cyan
}

$certificateExpiryDate = $startTime.AddYears($ExpiryYearsToSetForCertificate).ToString('yyyyMMdd')

# Create Certificate if required
$sqlCreateCertificate = @"
-- Create certificate on [$SqlInstanceToEncrypt]
declare @sql nvarchar(max);
declare @thumbprint varbinary(64);

select @thumbprint = c.thumbprint from sys.certificates c 
	where issuer_name = '$certificateSubject' and name = '$certificateName';

if @thumbprint is null
begin
	set @sql = 'use master; create certificate [$certificateName] with subject = ''$certificateSubject'', EXPIRY_DATE = ''$certificateExpiryDate'';';
	exec (@sql);
    print 'certificate created';
end
else
	print 'Certificate already exists';
"@

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Create Certificate.."
$sqlCreateCertificate + "`nGO`n`n" | Out-File -FilePath $scriptOutfile -Append
if($DryRun) {
    $sqlCreateCertificate | Write-Host -ForegroundColor Magenta
}
else {
    Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlCreateCertificate `
                    -MessagesToOutput -EnableException | Write-Host -ForegroundColor Cyan
}


# Backup current Certificate & expired Certificate if any
$sqlBackupCertificate = @"
-- Backup Certificate & Master Key on [$SqlInstanceToEncrypt]
use master;
go

declare @sql nvarchar(max);
exec xp_create_subdir '$backupDirectory';
go

/* **** If Cleanup of Files is required ****
DECLARE @cmd NVARCHAR(MAX);

SET @cmd = 'xp_cmdshell ''del "$masterKeyPath"''';
EXEC (@cmd); -- remove master key file

SET @cmd = 'xp_cmdshell ''del "$privateKeyPath"'''; 
EXEC (@cmd); -- remove private key file

SET @cmd = 'xp_cmdshell ''del "$certificatePath"'''; 
EXEC (@cmd); -- remove certificate file

$(if($hasCertificateExpired) {'--'}) /*
-- Remove expired certificate if any
SET @cmd = 'xp_cmdshell ''del "$expiredCertificatePath"''';
EXEC (@cmd); -- remove certificate file
$(if($hasCertificateExpired) {'--'}) */
*/
go

-- Backup master key
declare @sql nvarchar(max);
set @sql = 'use master; backup master key to file = ''$masterKeyPath'' encryption by password = ''$EncryptionPassword''';
exec (@sql);
go

-- Backup certificate
declare @sql nvarchar(max);
set @sql = 'use master; backup certificate [$certificateName]
	to file = ''$certificatePath''
	with private key (
		file = ''$privateKeyPath'',
		encryption by password = ''$EncryptionPassword''
	)';
exec (@sql);
go

$(if($hasCertificateExpired) {'--'}) /*
-- Backup expired certificate if any
declare @sql nvarchar(max);
set @sql = 'use master; backup certificate [$currentCertificateName]
	to file = ''$expiredCertificatePath''
	--with private key ( file = ''$privateKeyPath'', encryption by password = ''$EncryptionPassword'' )
    ';
exec (@sql)
$(if($hasCertificateExpired) {'--'}) */

/* IMPORTANT: COPY these files to some secure location Immediately before proceeding */

"@

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Backup certificate & Master Key to [$backupDirectory] on $SqlInstanceToEncrypt.."
$sqlBackupCertificate + "`nGO`n`n" | Out-File -FilePath $scriptOutfile -Append
if($DryRun) {
    $sqlBackupCertificate | Write-Host -ForegroundColor Magenta
}
else {
    Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlBackupCertificate `
                    -MessagesToOutput -EnableException | Write-Host -ForegroundColor Cyan
}


# List databases to Encrypt
$sqlGetAllUserDatabases = @"
/* Get list of databases to encrypt: $ClientName */
select [database_name] = d.name, d.state_desc, d.is_read_only, 
		d.is_in_standby, d.is_published, d.is_subscribed,
		rs.is_local, rs.replica_server_name, rs.role_desc
		,dm.mirroring_role_desc, d.is_encrypted,
		encryption_state_desc, percent_complete, --encryptor_thumbprint,
		[key_algorithm] = ek.key_algorithm + '_'+ convert(varchar,ek.key_length), 
		[certificate_name] = c.name, [certificate_expiry_date] = c.expiry_date, 
		[certificate_expiry_days] = case when c.expiry_date <= getdate() then 0 else datediff(day,getdate(),c.expiry_date) end,
		[certificate_subject] = c.subject,
		[certificate_start_date] = c.start_date,  c.pvt_key_last_backup_date,
		[encryption_key_create_date] = regenerate_date --encryption_scan_state_desc
from sys.databases d 
outer apply (select top 1 rs.is_local, ar.replica_server_name, rs.role_desc
			from sys.availability_replicas ar
			join sys.dm_hadr_availability_replica_states rs
			on rs.replica_id = ar.replica_id and rs.group_id = ar.group_id
			where ar.replica_id = d.replica_id
			) rs
outer apply (select top 1 dm.mirroring_role_desc
			from sys.database_mirroring dm
			where dm.database_id = d.database_id
				and dm.mirroring_role_desc is not null
			) dm
left join sys.dm_database_encryption_keys  ek
	on ek.database_id = d.database_id
left join sys.certificates c
	on c.thumbprint = ek.encryptor_thumbprint
    and c.pvt_key_encryption_type_desc = 'ENCRYPTED_BY_MASTER_KEY'
where 1=1
    and d.state_desc = 'ONLINE' 
    and is_read_only = 0 
    and d.database_id > 4
	$(if($hasCertificateExpired -or $pendingDbCount -gt 0 -or $expiredDbCount -gt 0){'--'})and d.is_encrypted = 0
	and d.source_database_id is null
"@

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Get list of dbs to Encrypt.."
$resultGetAllUserDatabases = @()
$resultGetAllUserDatabases += Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master `
                                -Query $sqlGetAllUserDatabases

$UserDatabasesToEncrypt = @()
$UserDatabasesToEncrypt += $resultGetAllUserDatabases | Where-Object {($_.role_desc -eq 'PRIMARY') -or ($_.mirroring_role_desc -eq 'PRINCIPAL') -or [String]::IsNullOrEmpty($_.role_desc)}
                                #Select-Object -ExpandProperty database_name

$hadrDatabases = @()
$hadrDatabases += $resultGetAllUserDatabases | Where-Object {([String]::IsNullOrEmpty($_.role_desc) -eq $false) -or ([String]::IsNullOrEmpty($_.mirroring_role_desc) -eq $false)}
                                #Select-Object -ExpandProperty database_name

#$resultGetAllUserDatabases | ogv -Title 'resultGetAllUserDatabases'
#$UserDatabasesToEncrypt | ogv -Title 'UserDatabasesToEncrypt'
#$hadrDatabases | ogv -Title 'hadrDatabases'

if($hadrDatabases.Count -gt 0) 
{
    # Scripts to Restore Certificate/Private Key
    "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Kindly use below query to Restore Certificate/Private Key.."
    $sqlRestoreCertificate = @"
USE [master];

/*  ****************************************************
    For Restore Scenario (AG/Mirroring/LogShipping/Backup-Restore)

    HADR Databases => $( ($hadrDatabases.database_name | % {"'$_'"}) -join ', ' )
*   **************************************************** */


-- Execute "Encrypt-SQLDatabases.ps1" for HADR Partner server also.
create master key encryption by /* Step 1: Unique to Destination Server */
	password = '<<Some Very Strong Password Here. Unique to each server>>';

-- For Restore Scenario (AG/Mirroring/LogShipping/Backup-Restore)
    -- Restore the certificates of each partner on every other partner before proceeding.
create certificate [$certificateName] /* Step 2: Details similar to Source Server */
	from file = '$certificatePath'
	with private key (
		file = '$privateKeyPath',
		decryption by password = '$EncryptionPassword'
	);
"@
    $horizontalLine + "`n" + $sqlRestoreCertificate + "`nGO`n`n" | Out-File -FilePath $scriptOutfile -Append
    $horizontalLine + "`n" + $sqlRestoreCertificate | Write-Host -ForegroundColor Magenta
}

# Loop through each database, and generate encryption key
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Loop through each database & generate Encryption Key.."
foreach($databaseObj in $UserDatabasesToEncrypt)
{
    $is_db_encrypted = $false
    [int]$certificate_expiry_days = 0
    $database = $databaseObj.database_name

    #Write-Debug "Inside loop"

    "`t$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Checking [$database] database.."

    # values which may not be present all the time
    if($databaseObj.is_encrypted) {
        $certificate_expiry_days = $databaseObj.certificate_expiry_days
        $is_db_encrypted = $databaseObj.is_encrypted
    }
    
    # If not encrypted, then create DEK
    if(-not $is_db_encrypted)
    { 
        $sqlCreateOrAlterEncryptionKey = @"
declare @sql nvarchar(max);
declare @thumbprint varbinary(64);

use [master];
-- Get certificate thumbprint
select @thumbprint = c.thumbprint from sys.certificates c 
	where issuer_name = '$certificateSubject' and name = '$certificateName';

use [$database];
-- Create encryption key
if not exists (select * from sys.dm_database_encryption_keys dek where dek.database_id = DB_ID()
					and dek.encryptor_type = 'CERTIFICATE' and dek.key_length = 128 
					and dek.encryptor_thumbprint = @thumbprint )
begin
	set @sql = 'use '+quotename(DB_NAME())+'; '
                +char(9)+'create database encryption key with algorithm = aes_128 '
	            +char(9)+char(9)+'encryption by server certificate [$certificateName];';
	exec (@sql);
    print 'Encryption Key created on [$database].'
end
else
	print 'Database Encryption key exists';
"@
        $sqlCreateOrAlterEncryptionKey + "`nGO`n`n" | Out-File -FilePath $scriptOutfile -Append
        if($DryRun) {
            $sqlCreateOrAlterEncryptionKey | Write-Host -ForegroundColor Magenta
        }
        else {
            Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlCreateOrAlterEncryptionKey `
                        -MessagesToOutput -EnableException | Write-Host -ForegroundColor Cyan
        }
    }
    else 
    {
        # If encrypted, then alter DEK
        if ($databaseObj.certificate_expiry_days -gt $ExpiryDaysThreshold) {
            $sqlCreateOrAlterEncryptionKey = "print 'Database [$database] is already encrypted with valid certificate.'";
        }
        else {
            $sqlCreateOrAlterEncryptionKey = @"
use [$database];

-- Alter encryption key
declare @sql nvarchar(max);
set @sql = 'use '+quotename(DB_NAME())+'; '
            +char(9)+'alter database encryption key '
	        +char(9)+char(9)+'encryption by server certificate [$certificateName];';
exec (@sql);
print 'Encryption Key certificate updated on [$database].'
"@
        }
    
        "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Working on [$database].."
        $sqlCreateOrAlterEncryptionKey + "`nGO`n`n" | Out-File -FilePath $scriptOutfile -Append
        if($DryRun) {
            $sqlCreateOrAlterEncryptionKey | Write-Host -ForegroundColor Magenta
        }
        else {
            Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlCreateOrAlterEncryptionKey `
                        -MessagesToOutput | Write-Host -ForegroundColor Cyan
        }
    }
}

# Loop through each database, and Encrypt it
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Loop through each database to Encrypt.."
foreach($databaseObj in $UserDatabasesToEncrypt)
{
    $is_db_encrypted = $false
    [int]$certificate_expiry_days = 0
    $database = $databaseObj.database_name

    # values which may not be present all the time
    if($databaseObj.is_encrypted) {
        $certificate_expiry_days = $databaseObj.certificate_expiry_days
        $is_db_encrypted = $databaseObj.is_encrypted
    }

    if($is_db_encrypted){
        "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Database [$database] is already encrypted."
    }
    else
    {
        $sqlEncryptDatabase = @"
-- Enable Encryption
use [master]; alter database [$database] set encryption on;
"@
    
        "$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Encrypting [$database].."
        $sqlEncryptDatabase + "`nGO`n" | Out-File -FilePath $scriptOutfile -Append
        if($DryRun) {
            $sqlEncryptDatabase | Write-Host -ForegroundColor Magenta
        }
        else {
            Invoke-DbaQuery -SqlInstance $conSqlInstanceToEncrypt -Database master -Query $sqlEncryptDatabase `
                        -MessagesToOutput | Write-Host -ForegroundColor Cyan
        }
    }
}


# Save Encryption Details on [CredentialManagerServer]
$sqlSaveEncryptionDetails = @"
-- Save Encryption Details of [$SqlInstanceToEncrypt]
-- Execute below on [$CredentialManagerServer].[$CredentialManagerDatabase]
if not exists (select * from dbo.tde_implementation_details where srv_name = '$SqlInstanceToEncrypt' and server_name = '$serverName' and server_host_name = '$serverHostName' and certificate_name = '$certificateName')
begin
	insert dbo.tde_implementation_details
    ( [srv_name], [at_server_name], [server_name], [domain], [server_host_name], [encrypted_databases], [total_database_count], [encryption_start_time], [encryption_end_time], [local_backup_directory], [remote_backup_directory], [certificate_name], [certificate_subject], [encryption_password], [certificate_file_path], [master_key_path], [private_key_path], [files_copied_to_remote] )
    select	[srv_name] ='$SqlInstanceToEncrypt', 
		    [at_server_name] = '$atServerName', 
		    [server_name] = '$serverName', 
		    [domain] = '$domain', 
		    [server_host_name] = '$serverHostName', 
		    [encrypted_databases] = $($UserDatabasesToEncrypt.Count), 
		    [total_database_count] = $($UserDatabasesToEncrypt.Count), 
		    [encryption_start_time] = SYSDATETIME(), 
		    [encryption_end_time] = null, 
		    [local_backup_directory] = '$backupDirectory', 
		    [remote_backup_directory] = '$RemoteBackupDirectory', 
		    [certificate_name] = '$certificateName', 
		    [certificate_subject] = '$certificateSubject', 
		    [encryption_password] = null, 
		    [certificate_file_path] = '$certificatePath', 
		    [master_key_path] = '$masterKeyPath', 
		    [private_key_path] = '$privateKeyPath', 
		    [files_copied_to_remote] = 0;
    
    print 'TDE Entry added for [$SqlInstanceToEncrypt].'
end
else
    print 'TDE Entry already exists for [$SqlInstanceToEncrypt].'
"@

"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Save encryption details of [$SqlInstanceToEncrypt] on [$CredentialManagerServer].[$CredentialManagerDatabase].[dbo].[tde_implementation_details]."
$horizontalLine + "`n" + $sqlSaveEncryptionDetails + "`nGO`n" | Out-File -FilePath $scriptOutfile -Append
$sqlSaveEncryptionDetails | Write-Host -ForegroundColor Magenta

Invoke-DbaQuery -SqlInstance $conCredentialManagerServer -Database $CredentialManagerDatabase -Query $sqlSaveEncryptionDetails `
                -MessagesToOutput | Write-Host -ForegroundColor Cyan



# Scripts to Check Encryption Status
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Kindly use below query to check Encryption Status.."
$sqlCheckEncryptionStatus = @"
USE [master];
-- Check Encryption Status
SELECT  --[sql_instance] = @@servername,
        [database_name] = d.name, encryption_state_desc, percent_complete, --encryptor_thumbprint,
		[key_algorithm] = ek.key_algorithm + '_'+ convert(varchar,ek.key_length), 
		[certificate_name] = c.name, [certificate_expiry_date] = c.expiry_date, 
		[certificate_expiry_days] = case when c.expiry_date <= getdate() then 0 else datediff(day,getdate(),c.expiry_date) end,
		[certificate_subject] = c.subject,
		[certificate_start_date] = c.start_date,  c.pvt_key_last_backup_date,
		[encryption_key_create_date] = regenerate_date --encryption_scan_state_desc
FROM sys.databases d
left join sys.dm_database_encryption_keys  ek
	on ek.database_id = d.database_id
left join sys.certificates c
	on c.thumbprint = ek.encryptor_thumbprint
    and c.pvt_key_encryption_type_desc = 'ENCRYPTED_BY_MASTER_KEY'
WHERE 1=1
and d.state_desc = 'ONLINE' 
and is_read_only = 0 
and d.database_id > 4
and d.source_database_id is null
"@
$horizontalLine + "`n" + $sqlCheckEncryptionStatus + "`nGO`n" | Out-File -FilePath $scriptOutfile -Append
$horizontalLine + "`n" + $sqlCheckEncryptionStatus | Write-Host -ForegroundColor Magenta


# Scripts to Restore Certificate/Private Key
"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Kindly use below query to Restore Certificate/Private Key.."
$sqlRestoreCertificate = @"
USE [master];
-- For Restore Scenario (AG/Mirroring/LogShipping/Backup-Restore)
create master key encryption by /* Step 1: Unique to Destination Server */
	password = '<<Some Very Strong Password Here. Unique to each server>>';

-- For Restore Scenario (AG/Mirroring/LogShipping/Backup-Restore)
create certificate [$certificateName] /* Step 2: Details similar to Source Server */
	from file = '$certificatePath'
	with private key (
		file = '$privateKeyPath',
		decryption by password = '$EncryptionPassword'
	);
"@
$horizontalLine + "`n" + $sqlRestoreCertificate + "`nGO`n" | Out-File -FilePath $scriptOutfile -Append
$horizontalLine + "`n" + $sqlRestoreCertificate | Write-Host -ForegroundColor Magenta


"$(Get-Date -Format yyyyMMMdd_HHmm) {0,-10} {1}" -f 'INFO:', "Opening scriptout file '$scriptOutfile'.."
notepad $scriptOutfile
#>

<#
Import-Module dbatools

# $allServerLoginCredential = Get-Credential -UserName 'sa' -Message 'All Server Login Credential'
$Server2Encrypt = 'AgHost-1A'

cls
E:\Github\SQLMonitor\Work-TDE\Encrypt-SQLDatabases.ps1 `
        -SqlInstanceToEncrypt $Server2Encrypt `
        -SqlCredential $allServerLoginCredential `
        -ScriptOut $False `
        #-EncryptionPassword 'YourMasterKeyPasswordHere' #-Debug
#>